In [40]:
from coinbase.wallet.client import Client 
import requests
import time 
import os
from dotenv import load_dotenv
import psycopg2
from datetime import datetime 
from email.utils import parsedate_to_datetime
import pytz


In [25]:
load_dotenv('./.env')


True

In [26]:
api_key = os.getenv("api_key")
api_secret = os.getenv("api_secret")
db_host = os.getenv("DB_HOST")
db_port = os.getenv("DB_PORT")
db_name = os.getenv("DB_NAME")
db_user = os.getenv("DB_USER")
db_password = os.getenv("DB_PASSWORD")


In [56]:
#creating connection to railway postgres db: 
connection = psycopg2.connect(
        host = db_host, 
        database = db_name, 
        user = db_user, 
        password = db_password, 
        port = db_port
    )

    #initializing cursor
cursor = connection.cursor() 

In [28]:
cursor.execute("SELECT 1;")

In [29]:
result = cursor.fetchone()

In [30]:
print(result) # connection is active 

(1,)


In [31]:
#establishing connection to coinbase API
client = Client(api_key, api_secret)

#pulling price data for ethereum and bitcoin 
price_eth = client.get_spot_price(currency_pair= "ETH-USD")
price_btc = client.get_spot_price(currency_pair= "BTC-USD")
price_near = client.get_spot_price(currency_pair= "NEAR-USD")

#adding timestamp to dictionaries 

#price_eth["datetime"] = datetime.now() 
#price_btc["datetime"] = datetime.now() 
 


In [62]:
def spot_price(client, c_pair): 
    return client.get_spot_price(currency_pair = c_pair)

In [63]:
coins = ['BTC-USD', 'ETH-USD', 'NEAR-USD']

In [64]:
prices = {i: spot_price(client, i) for i in coins}


In [65]:
prices["datetime"] = datetime.now()

In [67]:
prices['BTC-USD']

<APIObject @ 0x7fdf81b87100> {
  "amount": "34689.095",
  "base": "BTC",
  "currency": "USD"
}

In [33]:
btc_dict = {"datetime": datetime.now(), 
            "base": price_btc["base"],
            "amount": price_btc["amount"], 
            "currency": price_btc["currency"]}

eth_dict = {"datetime": datetime.now(), 
            "base": price_eth["base"],
            "amount": price_eth["amount"], 
            "currency": price_eth["currency"]}

near_dict = {"datetime": datetime.now(), 
            "base": price_near["base"],
            "amount": price_near["amount"], 
            "currency": price_near["currency"]}
            

In [34]:
near_dict

{'datetime': datetime.datetime(2023, 11, 7, 8, 19, 23, 828365),
 'base': 'NEAR',
 'amount': '1.459',
 'currency': 'USD'}

In [13]:
table_schema_btc = ("""
                    CREATE TABLE IF NOT EXISTS bitcoin
                    (id serial PRIMARY KEY, 
                     datetime timestamp, 
                     base varchar(10), 
                     amount numeric(10,4),
                     currency char(3));
                    """)

In [14]:
cursor.execute(table_schema_btc)

In [63]:
connection.commit()

In [15]:
insert_query = """ INSERT INTO bitcoin (datetime, base, amount, currency)
VALUES (%(datetime)s, %(base)s, %(amount)s, %(currency)s);
"""

In [16]:
cursor.execute(insert_query, btc_dict)

In [17]:
connection.commit()

In [23]:
#now trying ethereum table and insert statement 
table_schema_eth = ("""
                    CREATE TABLE IF NOT EXISTS ethereum
                    (id serial PRIMARY KEY, 
                     datetime timestamp, 
                     base char(3), 
                     amount numeric(10,4),
                     currency char(3));
                    """)

cursor.execute(table_schema_eth)

insert_query_eth = """ INSERT INTO ethereum (datetime, base, amount, currency)
VALUES (%(datetime)s, %(base)s, %(amount)s, %(currency)s);
"""

cursor.execute(insert_query_eth, eth_dict)

connection.commit() 


In [35]:
table_schema_near = ("""
                    CREATE TABLE IF NOT EXISTS near
                    (id serial PRIMARY KEY, 
                     datetime timestamp, 
                     base varchar(10), 
                     amount numeric(10,4),
                     currency char(3));
                    """)

In [68]:
client = Client(api_key, api_secret)
coins = ['BTC-USD', 'ETH-USD', 'NEAR-USD']

def spot_price(client, c_pair): 
        # Fetch the spot price for a given currency pair
        price_data = client.get_spot_price(currency_pair=c_pair)
        return price_data

prices = {i: spot_price(client, i) for i in coins}


In [71]:
prices["datetime"] = datetime.now()

In [75]:
prices.values()

dict_values([<APIObject @ 0x7fdf81bd74c0> {
  "amount": "34679.265",
  "base": "BTC",
  "currency": "USD"
}, <APIObject @ 0x7fdf81b61350> {
  "amount": "1858.71",
  "base": "ETH",
  "currency": "USD"
}, <APIObject @ 0x7fdf81bd7dd0> {
  "amount": "1.441",
  "base": "NEAR",
  "currency": "USD"
}, datetime.datetime(2023, 11, 7, 9, 0, 55, 189594)])

In [ ]:
table_schema_btc = ("""
                    CREATE TABLE IF NOT EXISTS bitcoin
                    (id serial PRIMARY KEY, 
                     datetime timestamp, 
                     base varchar(10), 
                     amount numeric(10,4),
                     currency char(3));
                    """)
        
        table_schema_eth = ("""
                    CREATE TABLE IF NOT EXISTS ethereum
                    (id serial PRIMARY KEY, 
                     datetime timestamp, 
                     base varchar(10), 
                     amount numeric(10,4),
                     currency char(3));
                    """)
        
        table_schema_near = ("""
                    CREATE TABLE IF NOT EXISTS near
                    (id serial PRIMARY KEY, 
                     datetime timestamp, 
                     base varchar(10), 
                     amount numeric(10,4),
                     currency char(3));
                    """)
        
        tables = [table_schema_btc, table_schema_eth, table_schema_near]

        #iterating through create table statements 
        for t in tables: 
            cursor.execute(t)
        
        connection.commit()

In [ ]:
cursor.execute("CREATE SCHEMA IF NOT EXISTS crypto;")
cursor.execute("GRANT ALL PRIVELEGS ON SCHEMA crypto to postgres;")
connection.commit() 